In [1]:
# Change directory to level up to import functions
import os
os.chdir('..')

#### Background
This analysis look at the Fantasy Premier League (FPL) performance of players from Championship teams, in their first season after being promoted to the Preimier League. See notes and caveats at the end.

The following areas are looked into:
* An exploratory data analysis of how these players perform.
  * Top performances from players in promoted teams
  * Top total team points from promoted teams
  * Championship goals and assits vs FPL Points in the following season
* A comparison of players from promoted teams with players from teams already in the league.
  * Overview of differences for each price point by position (link to ttest eli5/youtube vidoe)
  * £5.0m midfielders detail

#### Key Findings
* In general, avoid £5.0m midfielders from newly promoted teams; consider players from others teams instead.
* There is no statistically significant difference in player performance from promoted vs existing teams at other price points or positions, by different team strengths.
* Goals and assists in the Championship don’t always translate into Fantasy Premier League (FPL) points the following season.
* Patrick Bamford had a good season in 2020/21.

#### Notes and Caveats


Other things I assessed 
Experiemneted with different thresholds of definition for games played (mins/gws)
Built models to predict scores to see if championship flag was an important feature/significant. Similar restults to t tests.

Note this isn't looking only at players who played in the chapmionship the season before. It includes new signings that season.

We're only looking at players in teams promoted. 
Not players from promoted teams as that can be influnced by the strength of the club they are going to. 
E.g. When Tottenham signed Bale etc. This would be a seperate piece of analysis assesing the impact of signings 
from dif